In [0]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
sns.despine()

%matplotlib notebook

## Colab

In [2]:
from google.colab import drive
drive.mount('gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at gdrive


In [3]:
!git clone https://github.com/ne3x7/torchdiffeq.git

Cloning into 'torchdiffeq'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 227 (delta 24), reused 36 (delta 17), pack-reused 178
Receiving objects: 100% (227/227), 707.91 KiB | 2.87 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [0]:
from torchdiffeq.torchdiffeq import odeint_adjoint

In [5]:
!git clone https://github.com/ne3x7/fastai_reg.git
!cd fastai_reg && pip install -r requirements.txt

Cloning into 'fastai_reg'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 30689 (delta 8), reused 11 (delta 4), pack-reused 30671
Receiving objects: 100% (30689/30689), 433.24 MiB | 30.52 MiB/s, done.
Resolving deltas: 100% (22379/22379), done.
Checking out files: 100% (812/812), done.
Processing /content/fastai_reg
  Created wheel for fastai: filename=fastai-1.0.58.dev0-cp36-none-any.whl size=235865 sha256=88e59d269d385a381c6910b93fea4b169dea1550d90ff966ea88277bae0e6572
  Stored in directory: /tmp/pip-ephem-wheel-cache-rvznx28z/wheels/ab/71/c0/907b6fb10568c563ffff07e3d670debd4369d539f021db72f3
Successfully built fastai
  Found existing installation: fastai 1.0.57
    Uninstalling fastai-1.0.57:
      Successfully uninstalled fastai-1.0.57


In [0]:
from fastai.text import *

In [0]:
from fastai.basic_train import Learner, LearnerCallback

## Multi-file

In [0]:
def get_initial_values(P0, Q0, V0, angle0, theta0, Sn=2220, Vn=400, fn=60, Sb=100, Vb=400):
    """
    Initializes generator state vector `x` and network terminal variables vector `V`
    in machine base from given power flow values in system base and angle in radians.
    """
    
    # unpack parameters guess
    ws = 1
    KD = 0
    ra, x1d, xd, xq, H, T1d0, Kw, Tw, Tpss1, Tpss2, Tr, Tavr1, Tavr2, Te, K0 = theta0
    
    # define transforms
    
    # vf_MB = vf * Vb / Vn
    wb = 2 * np.pi * fn
    S_SBtoMB = Sb / Sn
    V_MBtoSB = Vn / Vb
    I_MBtoSB = Sn * Vb / (Sb * Vn)
    Z_MBtoSB = Sb * Vn ** 2 / (Sn * Vb ** 2)
    
    # initialize stator quantitites
    
    p0 = P0 / Sb
    q0 = Q0 / Sb
    Vt0 = V0 * np.exp(1j * angle0)
    S0 = p0 - 1j * q0
    I0 = S0 / Vt0.conjugate()
    vr0 = Vt0.real
    vi0 = Vt0.imag
    ir0 = -I0.real
    ii0 = -I0.imag
    
    # initialize DQ-quantities
    
    w0 = 1
    delta0 = np.angle(Vt0 + (ra + 1j * xq) * Z_MBtoSB * I0)
    
    Vdq0 = Vt0 * (1 / V_MBtoSB) * np.exp(1j * (-delta0 + np.pi/2))
    Idq0 = I0 * (1 / I_MBtoSB) * np.exp(1j * (-delta0 + np.pi/2))
    
    vd0 = Vdq0.real
    vq0 = Vdq0.imag
    id0 = Idq0.real
    iq0 = Idq0.imag
    
    # initialize order 3
    
    e1q0 = vq0 + ra * iq0 + x1d * id0
    
    # initialize AVR
    
    v = np.abs(Vt0)
    vref = v
    vs0 = 0
    vm0 = v
    vf0 = (e1q0 + (xd - x1d) * id0) * V_MBtoSB
    vr0 = K0 * (1 - Tavr1 / Tavr2) * (vref + vs0 - vm0)
    
    # initialize PSS
    
    v20 = 0
    
    # constants
    
    pm = (vq0 + ra * iq0) * iq0 + (vd0 + ra * id0) * id0
    vsmin, vsmax = -0.2, 0.2
    vfmin, vfmax = -6.4, 7
    
    # pack values
    
    x = np.array([delta0, w0, v20, vs0, vm0, vr0, vf0, e1q0])
    V = np.array([vd0, vq0, id0, iq0])
    c = np.array([pm, vsmin, vsmax, vfmin, vfmax, vref, vf0, vs0, wb, KD])
    
    return x, V, c, (p0, q0)

In [0]:
class RHSTrue(nn.Module):
    def __init__(self, c):
        super(RHSTrue, self).__init__()
        
        self.theta = torch.from_numpy(np.array([0.003, 0.3, 1.81, 1.76, 3.5, 8.,
                                                10., 10., 0.05, 0.02, 0.015, 1.,
                                                1., 0.0001, 200]))
        self.c = c
        
    def forward(self, t, x, v):
        vd = v[:, 0] * torch.sin(x[:, 0] - v[:, 1])
        vq = v[:, 0] * torch.cos(x[:, 0] - v[:, 1])

        id = (self.theta[3] * x[:, 7] - self.theta[3] * vq - self.theta[0] * vd) / (self.theta[1] * self.theta[3] + self.theta[0] ** 2)
        iq = (self.theta[0] * x[:, 7] - self.theta[0] * vq + self.theta[1] * vd) / (self.theta[1] * self.theta[3] + self.theta[0] ** 2)

        p = 22.2 * (vd * id + vq * iq)
        q = 22.2 * (vq * id - vd * iq)
        
        pe = (vq + self.theta[0] * iq) * iq + (vd + self.theta[0] * id) * id

        return torch.stack([
            self.c[8] * (x[:, 1] - 1),
            (self.c[0] - pe) / (2 * self.theta[4]),
            self.theta[6] * (self.c[0] - pe) / (2 * self.theta[4]) - x[:, 2] / self.theta[7],
            (self.theta[8] * (self.theta[6] * (self.c[0] - pe)
                              / (2 * self.theta[4]) - x[:, 2]
                              / self.theta[7]) + x[:, 2] - x[:, 3]) / self.theta[9],
            (v[:, 0] - x[:, 4]) / self.theta[10],
            (self.theta[14] * (1 - self.theta[11] / self.theta[12]) * (self.c[5] + x[:, 3] - x[:, 4]) - x[:, 5]) / self.theta[12],
            ((x[:, 5] + self.theta[14] * self.theta[11] * (self.c[5] + x[:, 3] - x[:, 4])
              / self.theta[12] + self.c[6]) * (1 + self.c[7] * (v[:, 0] / x[:, 4] - 1)) - x[:, 6]) / self.theta[13],
            (- x[:, 7] - (self.theta[2] - self.theta[1]) * id + x[:, 6]) / self.theta[5]
        ], dim=1)

In [0]:
class RHS(nn.Module):
    def __init__(self, c):
        super(RHS, self).__init__()
        
        self.theta = nn.Parameter(torch.rand(15))
        self.s = torch.from_numpy(np.array([0.01, 1., 10., 10., 10., 10.,
                                            10., 10., 0.1, 0.1, 0.1, 1.,
                                            1., 0.001, 1000]))
        self.c = c
        
    def forward(self, t, x, v):
        vd = v[:, 0] * torch.sin(x[:, 0] - v[:, 1])
        vq = v[:, 0] * torch.cos(x[:, 0] - v[:, 1])

        id = (self.s[3] * self.theta[3] * x[:, 7] - self.s[3] * self.theta[3] * vq - self.s[0] * self.theta[0] * vd) / (self.s[1] * self.theta[1] * self.s[3] * self.theta[3] + (self.s[0] * self.theta[0]) ** 2)
        iq = (self.s[0] * self.theta[0] * x[:, 7] - self.s[0] * self.theta[0] * vq + self.s[1] * self.theta[1] * vd) / (self.s[1] * self.theta[1] * self.s[3] * self.theta[3] + (self.s[0] * self.theta[0]) ** 2)
        p = 22.2 * (vd * id + vq * iq)
        q = 22.2 * (vq * id - vd * iq)
        
        pe = (vq + self.s[0] * self.theta[0] * iq) * iq + (vd + self.s[0] * self.theta[0] * id) * id

        return torch.stack([
            self.c[8] * (x[:, 1] - 1),
            (self.c[0] - pe) / (2 * self.s[4] * self.theta[4]),
            self.s[6] * self.theta[6] * (self.c[0] - pe) / (2 * self.s[4] * self.theta[4]) - x[:, 2] / (self.s[7] * self.theta[7]),
            (self.s[8] * self.theta[8] * (self.s[6] * self.theta[6] * (self.c[0] - pe)
                              / (2 * self.s[4] * self.theta[4]) - x[:, 2]
                              / (self.s[7] * self.theta[7])) + x[:, 2] - x[:, 3]) / (self.s[9] * self.theta[9]),
            (v[:, 0] - x[:, 4]) / (self.s[10] * self.theta[10]),
            (self.s[14] * self.theta[14] * (1 - self.s[11] * self.theta[11] / (self.s[12] * self.theta[12])) * (self.c[5] + x[:, 3] - x[:, 4]) - x[:, 5]) / (self.s[12] * self.theta[12]),
            ((x[:, 5] + self.s[14] * self.theta[14] * self.s[11] * self.theta[11] * (self.c[5] + x[:, 3] - x[:, 4])
              / (self.s[12] * self.theta[12]) + self.c[6]) * (1 + self.c[7] * (v[:, 0] / x[:, 4] - 1)) - x[:, 6]) / (self.s[13] * self.theta[13]),
            (- x[:, 7] - (self.s[2] * self.theta[2] - self.s[1] * self.theta[1]) * id + x[:, 6]) / (self.s[5] * self.theta[5])
        ], dim=1)

In [0]:
class RHSCUDA(nn.Module):
    def __init__(self, c):
        super(RHSCUDA, self).__init__()
        
        self.theta = nn.Parameter(torch.rand(15))
        self.s = torch.from_numpy(np.array([0.01, 1., 10., 10., 10., 10.,
                                            10., 10., 0.1, 0.1, 0.1, 1.,
                                            1., 0.001, 1000])).cuda()
        self.c = c
        
    def forward(self, t, x, v):
        vd = v[:, 0] * torch.sin(x[:, 0] - v[:, 1])
        vq = v[:, 0] * torch.cos(x[:, 0] - v[:, 1])

        id = (self.s[3] * self.theta[3] * x[:, 7] - self.s[3] * self.theta[3] * vq - self.s[0] * self.theta[0] * vd) / (self.s[1] * self.theta[1] * self.s[3] * self.theta[3] + (self.s[0] * self.theta[0]) ** 2)
        iq = (self.s[0] * self.theta[0] * x[:, 7] - self.s[0] * self.theta[0] * vq + self.s[1] * self.theta[1] * vd) / (self.s[1] * self.theta[1] * self.s[3] * self.theta[3] + (self.s[0] * self.theta[0]) ** 2)
        p = 22.2 * (vd * id + vq * iq)
        q = 22.2 * (vq * id - vd * iq)
        
        pe = (vq + self.s[0] * self.theta[0] * iq) * iq + (vd + self.s[0] * self.theta[0] * id) * id

        return torch.stack([
            self.c[8] * (x[:, 1] - 1),
            (self.c[0] - pe) / (2 * self.s[4] * self.theta[4]),
            self.s[6] * self.theta[6] * (self.c[0] - pe) / (2 * self.s[4] * self.theta[4]) - x[:, 2] / (self.s[7] * self.theta[7]),
            (self.s[8] * self.theta[8] * (self.s[6] * self.theta[6] * (self.c[0] - pe)
                              / (2 * self.s[4] * self.theta[4]) - x[:, 2]
                              / (self.s[7] * self.theta[7])) + x[:, 2] - x[:, 3]) / (self.s[9] * self.theta[9]),
            (v[:, 0] - x[:, 4]) / (self.s[10] * self.theta[10]),
            (self.s[14] * self.theta[14] * (1 - self.s[11] * self.theta[11] / (self.s[12] * self.theta[12])) * (self.c[5] + x[:, 3] - x[:, 4]) - x[:, 5]) / (self.s[12] * self.theta[12]),
            ((x[:, 5] + self.s[14] * self.theta[14] * self.s[11] * self.theta[11] * (self.c[5] + x[:, 3] - x[:, 4])
              / (self.s[12] * self.theta[12]) + self.c[6]) * (1 + self.c[7] * (v[:, 0] / x[:, 4] - 1)) - x[:, 6]) / (self.s[13] * self.theta[13]),
            (- x[:, 7] - (self.s[2] * self.theta[2] - self.s[1] * self.theta[1]) * id + x[:, 6]) / (self.s[5] * self.theta[5])
        ], dim=1)

In [0]:
class ODE(nn.Module):
    def __init__(self, func, dim, t, x0):
        super(ODE, self).__init__()

        self.rhs = func
        self.dim = dim
        self.t = t
        self.y0 = x0
    
    def forward(self, x):
        y_exog = x.transpose(0, 1) # (T, M, D)
        preds = odeint_adjoint(self.rhs, self.y0, y_exog, self.t, method='euler') # (T, M, D)
        pqs = []
        
        for delta, e1q, vt, phi in zip(preds[:, :, 0], preds[:, :, -1], y_exog[:, :, 0], y_exog[:, :, 1]):
            vd = vt * torch.sin(delta - phi)
            vq = vt * torch.cos(delta - phi)
            
            denom = self.rhs.s[1] * self.rhs.theta[1] * self.rhs.s[3] * self.rhs.theta[3] \
            + (self.rhs.s[0] * self.rhs.theta[0]) ** 2

            id = (self.rhs.s[3] * self.rhs.theta[3] * e1q - self.rhs.s[3] * self.rhs.theta[3] * vq \
                  - self.rhs.s[0] * self.rhs.theta[0] * vd) / denom
            iq = (self.rhs.s[0] * self.rhs.theta[0] * e1q - self.rhs.s[0] * self.rhs.theta[0] * vq \
                  + self.rhs.s[1] * self.rhs.theta[1] * vd) / denom
            
            p = 22.2 * (vd * id + vq * iq) # (M, )
            q = 22.2 * (vq * id - vd * iq) # (M, )
            
            pqs.append(torch.stack([p, q], dim=1)) # (M, D)
            
        ans = torch.stack(pqs, dim=0).transpose(0, 1)
        return ans

In [0]:
class ODE2(nn.Module):
    def __init__(self, func, dim, t, x0):
        super(ODE2, self).__init__()

        self.rhs = func
        self.dim = dim
        self.t = t
        self.y0 = x0
    
    def forward(self, x):
        y_exog = x.transpose(0, 1) # (T, M, D)
        preds = odeint_adjoint(self.rhs, self.y0, y_exog, self.t,
                               method='euler', options={'step_size': 1e-4}) # (T, M, D)
        pqs = []
        
        for delta, e1q, vt, phi in zip(preds[:, :, 0], preds[:, :, -1], y_exog[:, :, 0], y_exog[:, :, 1]):
            vd = vt * torch.sin(delta - phi)
            vq = vt * torch.cos(delta - phi)
            
            denom = self.rhs.s[1] * self.rhs.theta[1] * self.rhs.s[3] * self.rhs.theta[3] \
            + (self.rhs.s[0] * self.rhs.theta[0]) ** 2

            id = (self.rhs.s[3] * self.rhs.theta[3] * e1q - self.rhs.s[3] * self.rhs.theta[3] * vq \
                  - self.rhs.s[0] * self.rhs.theta[0] * vd) / denom
            iq = (self.rhs.s[0] * self.rhs.theta[0] * e1q - self.rhs.s[0] * self.rhs.theta[0] * vq \
                  + self.rhs.s[1] * self.rhs.theta[1] * vd) / denom
            
            p = 22.2 * (vd * id + vq * iq) # (M, )
            q = 22.2 * (vq * id - vd * iq) # (M, )
            
            pqs.append(torch.stack([p, q], dim=1)) # (M, D)
            
        ans = torch.stack(pqs, dim=0).transpose(0, 1)
        return ans

In [0]:
class ParamsClampCallback(LearnerCallback):
    def __init__(self, learn:Learner, clip:float=None):
        super().__init__(learn)
        self.clip = clip
        
    def on_batch_end(self, **kwargs):
        if self.clip is not None:
            for p in self.learn.model.parameters(): p.data.clamp_(0, self.clip)

In [0]:
class GenLearnLearner(Learner):
    def __init__(self, data:DataBunch, model:nn.Module, clip:float=None,
                 params_clip:float=None, **learn_kwargs):
        super().__init__(data, model, **learn_kwargs)
        if clip: self.callback_fns.append(partial(GradientClipping, clip=clip))
        if params_clip:
            cb = ParamsClampCallback(self, clip=params_clip)
            self.callbacks.append(cb)

In [0]:
PATH = 'gdrive/My Drive/Generator Learning/simple/std_params/R_e1'
fnames = os.listdir(PATH)
n_samples = len(fnames)

In [0]:
x0, v0, c, (p0, q0) = get_initial_values(P0=1997.9999999936396,
                                         Q0=967.9249699065775,
                                         V0=1.0,
                                         angle0=0.494677176989154,
                                         theta0=np.array([0.003, 0.3, 1.81, 1.76, 3.5, 8.,
                                                          10., 10., 0.05, 0.02, 0.015, 1.,
                                                          1., 0.0001, 200]))

In [0]:
data_size = 150001
items = []
for fname in fnames:
    if fname == '.DS_Store': continue
    df = pd.read_csv(f'{PATH}/{fname}')
    df = df.drop_duplicates('t')
    df = df.drop(['t'], axis=1)
    items.append(df.values[:data_size])
items = np.stack(items, axis=0)
n_samples, n_meas, n_vars = items.shape
# items_sparse = items[:, ::1000, :]
#items_interp = np.stack(
#    [np.stack(
#        [np.interp(range(150001), range(0, 150001, 1000), elem[::1000]) for elem in v_], axis=0
#    ) for v_ in items.transpose(2, 0, 1)], axis=0
#).transpose(1, 2, 0)

In [0]:
bs=10

In [0]:
data_x, data_y = items[:, :, :2], items[:, :, 2:]
x = ItemList(data_x, label_cls=ItemList)._label_from_list(data_y, from_item_lists=True)
x.y.items = x.y.items.astype(np.float64)
bunch = PhysGenLearnDataBunch.create(train_ds=x, valid_ds=None, bs=bs)

In [0]:
t = torch.linspace(0, 15, steps=data_size)
initial_x = np.repeat(x0.reshape((1, -1)), bs, 0)

In [0]:
rhs = RHSCUDA(c).double().cuda()
ode = ODE(rhs, 2, t.cuda(), torch.from_numpy(initial_x).cuda()).double().cuda()

In [0]:
lrn = GenLearnLearner(bunch, ode, loss_func=nn.MSELoss(), metrics=[], clip=1.0, params_clip=1.0)

In [0]:
lrn.lr_find()
lrn.recoder.plot(suggestion=True)

epoch,train_loss,valid_loss,time


In [0]:
def get_batch(time_span):
    i = torch.from_numpy(np.random.choice(np.arange(n_samples - batch_size, dtype=np.int64), batch_size, replace=False))
    s = torch.from_numpy(np.random.choice(np.arange(data_size - time_span, dtype=np.int64), batch_size, replace=False))
    batch_y0 = true_y[s, i]  # (M, D)
    batch_t = t[:time_span]  # (T)
    batch_y = torch.stack([true_y[s + j, i] for j in range(time_span)], dim=0)  # (T, M, D)
    batch_y_exog = torch.stack([exog_y[s + j, i] for j in range(time_span)], dim=0)  # (T, M, D)
    return batch_y0, batch_t, batch_y, batch_y_exog

def get_batch_sparse(time_span):
    i = torch.from_numpy(np.random.choice(np.arange(n_samples - batch_size, dtype=np.int64), batch_size, replace=False))
    s = torch.from_numpy(np.random.choice(np.arange(data_size - time_span, dtype=np.int64), batch_size, replace=False))
    batch_y0 = true_y[s, i]  # (M, D)
    batch_t = t[:time_span]  # (T)
    batch_y = torch.stack([true_y[s + j, i] for j in range(time_span)], dim=0)  # (T, M, D)
    batch_y_exog = torch.stack([exog_y_sparse[s + j, i] for j in range(time_span)], dim=0)  # (T, M, D)
    return batch_y0, batch_t, batch_y, batch_y_exog

In [0]:
# rhs = RHS(c).double()
optimizer = optim.Adam(rhs.parameters())

In [0]:
time_schedule = [1000] * 500 + [5000] * 200 + [10000] * 100
# saved_value = np.inf
for iter_n, tspan in zip(list(range(800)), time_schedule):
    theta_prev = rhs.theta.clone()
    optimizer.zero_grad()
    batch_y0, batch_t, batch_y, batch_y_exog = get_batch(tspan)
    pred_y = odeint_adjoint(rhs, batch_y0, batch_y_exog, batch_t, method='euler')
    loss = ((pred_y - batch_y) / torch.from_numpy(np.array([10., 1., 0.001,
                                                            0.001, 10., 1., 10., 10.]))).pow(2).mean()
    loss.backward()
    optimizer.step()
    for param in rhs.parameters():
        param.data.clamp_(min=1e-1, max=1)
    reference = (rhs.theta - torch.from_numpy(np.array([0.003, 0.3, 1.81, 1.76, 3.5, 8.,
                                                        10., 10., 0.05, 0.02, 0.015, 1.,
                                                        1., 0.0001, 200]))
                 / rhs.s).pow(2).sum().sqrt().item()
    # dist = (rhs.theta - theta_prev).pow(2).sum().sqrt().item()
    print('[%04d]\tloss\t%e\tdistance to solution\t%e' % (iter_n, loss.item(), reference))
    if reference < saved_value:
        torch.save(rhs.state_dict(), 'model.pth')
        !cp model.pth gdrive/My\ Drive/Generator\ Learning/simple/model.mdl
        saved_value = reference

[0000]	loss	1.898313e-03	distance to solution	1.474812e+00
[0001]	loss	1.761428e-03	distance to solution	1.474205e+00
[0002]	loss	1.455444e-03	distance to solution	1.473457e+00
[0003]	loss	8.655249e-04	distance to solution	1.472761e+00
[0004]	loss	1.481869e-03	distance to solution	1.472098e+00
[0005]	loss	2.075205e-03	distance to solution	1.471556e+00
[0006]	loss	7.809726e-04	distance to solution	1.471024e+00
[0007]	loss	1.885452e-03	distance to solution	1.470430e+00
[0008]	loss	1.883457e-03	distance to solution	1.469728e+00
[0009]	loss	2.010967e-03	distance to solution	1.468954e+00
[0010]	loss	2.564128e-03	distance to solution	1.468104e+00
[0011]	loss	1.204515e-03	distance to solution	1.467251e+00
[0012]	loss	1.398787e-03	distance to solution	1.466467e+00
[0013]	loss	1.394764e-03	distance to solution	1.465781e+00
[0014]	loss	9.552815e-04	distance to solution	1.465151e+00
[0015]	loss	1.892327e-03	distance to solution	1.464535e+00
[0016]	loss	1.630768e-03	distance to solution	1.463827e+

## Sparse

In [0]:
data_size = 150001
data_size_sparse = 151
batch_size = 50

In [0]:
all_df = []
for fname in fnames:
    try:
        df = pd.read_csv(f'{PATH}/{fname}')
        df = df.drop_duplicates('t')
        df = df.drop(['t'], axis=1)
        all_df.append(df.values[:data_size])
    except Exception:
        print(fname)
all_df = np.array(all_df).transpose(1, 0, 2)
pq = all_df[:, :, [2, 3]]
v = all_df[:, :, [0, 1]]
v_sparse = np.stack(
    [np.stack(
        [np.interp(range(150001), range(0, 150001, 1000), elem[::1000]) for elem in v_], axis=0
        # [interp1d(range(0, 150001, 1000), elem[::1000], 'cubic')(range(150001)) for elem in v_], axis=0
    ) for v_ in v.transpose(2, 1, 0)], axis=0
).transpose(2, 1, 0)
del all_df

In [0]:
v_sparse = np.stack(
    [np.stack(
        # [np.interp(range(150001), range(0, 150001, 1000), elem[::1000]) for elem in v_], axis=0
        [interp1d(range(0, 150001, 1000), elem[::1000], 'linear')(range(150001)) for elem in v_], axis=0
    ) for v_ in v.transpose(2, 1, 0)], axis=0
).transpose(2, 1, 0)

In [0]:
initial_x = np.repeat(x0.reshape((1, -1)), n_samples, 0)
t = torch.linspace(0, 15, steps=data_size)
exog_y = torch.from_numpy(v)
exog_y_sparse = torch.from_numpy(v_sparse)

In [0]:
# with torch.no_grad():
#     rhs_true = RHSTrue(c)
#     true_y = odeint_adjoint(rhs_true, torch.from_numpy(initial_x), torch.from_numpy(v), t, method='euler')
with torch.no_grad():
    rhs_true = RHSTrue(c)
    true_y_sparse = odeint_adjoint(rhs_true, torch.from_numpy(initial_x), torch.from_numpy(v_sparse), t, method='euler')

In [0]:
i = -1 # batch index 0 to 99
j = 1 # 0 for P, 1 for Q
deltas = true_y_sparse[:, i, 0]
e1qs = true_y_sparse[:, i, -1]
pqs = []
for delta, e1q, vt, phi in zip(deltas, e1qs, v_sparse[:, i, 0], v_sparse[:, i, 1]):
    vd = vt * np.sin(delta - phi)
    vq = vt * np.cos(delta - phi)
    
    id = (1.76 * e1q - 1.76 * vq - 0.003 * vd) / (0.3 * 1.76 + 0.003 ** 2)
    iq = (0.003 * e1q - 0.003 * vq + 0.3 * vd) / (0.3 * 1.76 + 0.003 ** 2)
    
    p = 22.2 * (vd * id + vq * iq)
    q = 22.2 * (vq * id - vd * iq)
    pqs.append([p, q])
pqs = np.array(pqs)
plt.plot(pq[::1000, i, j])
plt.plot(pqs[::1000, j], '--')
plt.savefig('fig.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [0]:
# interpolation
np.sum(np.sqrt(np.sum((pq[:, i, :] - pqs) ** 2, axis=1))) / np.sum(np.sqrt(np.sum(pq[:, i, :] ** 2, axis=1)))

0.0003864066520109236

In [0]:
# solution
np.sum(np.sqrt(np.sum((pq[:, i, :] - pqs) ** 2, axis=1))) / np.sum(np.sqrt(np.sum(pq[:, i, :] ** 2, axis=1)))

1.7561617292668732e-06

In [0]:
# interpolation only keyvals
np.sum(np.sqrt(np.sum((pq[::1000, i, :] - pqs[::1000]) ** 2, axis=1))) / np.sum(np.sqrt(np.sum(pq[::1000, i, :] ** 2, axis=1)))

0.00036306381738783575

In [0]:
# sp interpolation
np.sum(np.sqrt(np.sum((pq[:, i, :] - pqs) ** 2, axis=1))) / np.sum(np.sqrt(np.sum(pq[:, i, :] ** 2, axis=1)))

0.0003864066520109236

In [0]:
# sp interpolation only keyvals
np.sum(np.sqrt(np.sum((pq[::1000, i, :] - pqs[::1000]) ** 2, axis=1))) / np.sum(np.sqrt(np.sum(pq[::1000, i, :] ** 2, axis=1)))

0.00036306381738783575

In [0]:
from scipy.interpolate import interp1d, interp2d, interpnd